In [1]:
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from geopy.geocoders import Nominatim
%matplotlib inline 

## get crashes_df from the API and clean

In [2]:
endpoint_crashes = 'https://data.cityofchicago.org/resource/85ca-t3if.geojson?$select=crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,first_crash_type,trafficway_type,lane_cnt,road_defect,prim_contributory_cause,sec_contributory_cause,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,street_no,street_direction,street_name,latitude,longitude,location&$limit=650000'
res = requests.get(endpoint_crashes)
res = res.json()

In [3]:
crashes_df = gpd.GeoDataFrame.from_features(res['features'])

In [ ]:
crashes_df.info()

In [ ]:
type(crashes_df)

In [4]:
#filter out crashes where there was debris on roadway
crashes_df = crashes_df.loc[crashes_df['road_defect'] != 'DEBRIS ON ROADWAY']

In [5]:
#filter out crashes where the traffic control device was functioning improperly or not functioning
device_not_working = ['FUNCTIONING IMPROPERLY','NOT FUNCTIONING']
crashes_df = crashes_df.loc[~crashes_df['device_condition'].isin(device_not_working)]
#crashes.loc[crashes['crash_record_id'].isin(crash_id_peds)]

In [6]:
#there is one row in there with POINT of (0,0) and it throws off plotting the map later on, so get rid of it
crashes_df = crashes_df[crashes_df['geometry'].x != 0]

In [7]:
#concat full address
crashes_df['full_address'] = crashes_df['street_no'] + ' ' + crashes_df['street_direction'] + ' ' + crashes_df['street_name'] + ' CHICAGO IL'

In [8]:
#filter out 5 rows with null full_address
crashes_df = crashes_df[crashes_df['full_address'].notna()]

In [9]:
#how many null geometries are there? 3877
len(crashes_df[crashes_df['geometry'].isna()])

3877

In [10]:
#geopy code. this takes an address and generate long/lat
#checking to see if it works for one single row before applying to larger set
geolocator = Nominatim(user_agent="colin")
location = geolocator.geocode(crashes_df.loc[1,'full_address'])
print(location.address)
print((location.longitude, location.latitude))

3410, South Damen Avenue, Ducktown, McKinley Park, Chicago, Cook County, Illinois, 60608, United States
(-87.67542055179473, 41.83173120922617)


In [ ]:
crashes_df.loc[1]

In [11]:
#check what data type is generated
type(location.longitude)

float

In [ ]:
crashes_df.loc[0,'full_address']

In [12]:
#could try counter, progress bar
#this code looks at every row of crashes_df. it tries to generate long/lat using the full_address. if it can't, it prints the index. if it can, it makes it into a POINT geometry and puts it in the geometry column in the row it got the address from
for index, row in crashes_df.iterrows():
    if row.geometry is None:
        try:
            geolocator = Nominatim(user_agent="colin")
            location = geolocator.geocode(row['full_address'])
            crashes_df.at[index, 'geometry'] = Point((location.longitude, location.latitude))
        except: 
            print(index)

134
140
362
461
623
703
734
956
1196
1301
1309
1836
1943
1984
2965
3135
3243
3334
3462
3538
3583
3612
4028
4081
4231
4800
4845
4942
5110
5229
5308
5483
5556
5685
5839
5888
6323
6444
6459
6585
6654
6776
6820
6868
7080
7207
7418
7476
7642
7676
8225
8262
8374
8400
8525
8929
8955
9102
9324
9579
9715
10122
10316
10508
10530
11141
11673
11681
11732
11907
12036
12091
12114
12584
12588
12917
13227
13341
13416
13618
13685
13901
14330
14465
14472
14510
14524
14538
15153
15162
15482
15549
15575
15581
15795
15815
15819
16078
16326
16335
16380
16780
16813
16984
17063
17179
18062
18150
18243
18367
18475
18488
18807
19322
19344
19586
19804
19944
19998
20191
20462
21265
21351
21384
21730
22802
22967
23070
23220
24701
24703
24911
25099
25772
25781
26151
26693
27066
27103
27241
27282
27550
27611
27655
28579
28674
29005
29235
29423
29675
29725
29849
29911
29919
30581
30602
30926
31151
31332
31344
31391
31709
31710
31734
31841
31856
32215
32276
32525
32895
32933
33275
33356
33652
33876
33879
33917
34380
3

276274
276566
276792
277281
277292
277556
277571
277758
277829
278195
278867
278966
279522
279937
280246
280254
280463
280505
280562
280597
280734
280793
280794
280931
281182
281191
281631
281953
282024
282682
283564
284170
284243
284276
284534
284806
284808
285058
285117
285135
285328
285906
285926
286135
286148
286211
286330
286353
286407
286432
286451
286474
286740
286765
286772
287236
287280
287317
287324
287357
287402
287404
287423
288215
288287
288640
288684
288832
288888
289025
289078
289080
289081
289157
289165
289277
289301
289312
289348
289377
289431
289613
290438
290455
290503
290660
290826
290868
290986
290998
291850
291915
292247
292267
292507
292640
292815
292844
293075
293293
293354
293524
293725
293789
293833
293889
293946
293949
294113
294116
294306
294400
294490
294522
294888
294906
294911
294952
295351
295544
295586
295665
295822
295839
295984
295987
296044
296142
296182
296230
296243
296386
296603
296812
296880
297044
297330
297416
297419
297505
298072
298297
298478

In [13]:
#how many null geometries are there now that we populated missing ones from the address?
#there are still 2097. that must be how many weird addresses there are that it can't generate a lat/long for.
#addresses like '1 w parking lot a' 
crashes_df[crashes_df['geometry'].isna()]

,geometry,injuries_fatal,injuries_non_incapacitating,crash_record_id,injuries_incapacitating,injuries_no_indication,latitude,street_no,injuries_unknown,device_condition,...,first_crash_type,injuries_reported_not_evident,most_severe_injury,prim_contributory_cause,sec_contributory_cause,street_direction,posted_speed_limit,injuries_total,lane_cnt,full_address
134,None,0,0,69343adf26ccf13cd11e6efef9b4120a52e3bf3098caff...,0,2,None,2,0,NO CONTROLS,...,SIDESWIPE OPPOSITE DIRECTION,0,NO INDICATION OF INJURY,"OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELE...",WEATHER,W,25,0,None,2 W TERMINAL ST CHICAGO IL
140,None,0,0,df3c10c5cae0e932d0d17d89ee54d0aad2bb208685668f...,0,2,None,6701,0,FUNCTIONING PROPERLY,...,SIDESWIPE SAME DIRECTION,1,"REPORTED, NOT EVIDENT",FAILING TO REDUCE SPEED TO AVOID CRASH,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,S,45,1,None,6701 S CHICAGO SKYWAY IB CHICAGO IL
362,None,0,0,4735f0097597b820d0a726d7ca37f8fe9e436b027af957...,0,8,None,8701,0,FUNCTIONING PROPERLY,...,REAR TO FRONT,0,NO INDICATION OF INJURY,IMPROPER BACKING,DISREGARDING OTHER TRAFFIC SIGNS,S,35,0,None,8701 S CHICAGO SKYWAY IB CHICAGO IL
461,None,0,0,c3248f793a0d840e04922eec000984d8ed74370a42c173...,0,2,None,2,0,UNKNOWN,...,PEDESTRIAN,0,NO INDICATION OF INJURY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,25,0,None,2 W TERMINAL ST CHICAGO IL
623,None,0,0,69a5d35a7f1d8253f95734b294ef5f552ac8ed5c6a711a...,0,2,None,710,0,FUNCTIONING PROPERLY,...,REAR END,0,NO INDICATION OF INJURY,FOLLOWING TOO CLOSELY,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,W,45,0,None,710 W OHARE ST CHICAGO IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637626,None,0,0,f3d7e90cc0af47e25c44f4c3dca066e24670c020e476ac...,0,3,None,3,0,NO CONTROLS,...,SIDESWIPE SAME DIRECTION,0,NO INDICATION OF INJURY,UNABLE TO DETERMINE,NOT APPLICABLE,W,15,0,None,3 W TERMINAL ST CHICAGO IL
637753,None,0,0,21f7f94ba3a5c2e044b137c9e0a7d42e28698df0bbc00d...,0,2,None,3307,0,NO CONTROLS,...,PARKED MOTOR VEHICLE,0,NO INDICATION OF INJURY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,30,0,1,3307 W FRANKLIN SD CHICAGO IL
638017,None,0,0,477ceadd190b5de622626b7218c1ff855384357c6cf6fd...,0,2,None,8800,0,NO CONTROLS,...,SIDESWIPE SAME DIRECTION,0,NO INDICATION OF INJURY,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,S,50,0,4,8800 S CHICAGO SKYWAY OB CHICAGO IL
638731,None,0,0,0cfb3fa8cb34427cf0f5debe87d0ca96326cba533dd62a...,0,1,None,4747,0,NO CONTROLS,...,PARKED MOTOR VEHICLE,0,NO INDICATION OF INJURY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,S,30,0,1,4747 S DR MARTIN LUTHER KING JR SD CHICAGO IL


In [14]:
#remove null geometries that we could not generate a POINT for from the address
crashes_df = crashes_df[crashes_df['geometry'].notna()]

In [15]:
crashes_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 631405 entries, 0 to 639252
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   geometry                       631405 non-null  geometry
 1   injuries_fatal                 630078 non-null  object  
 2   injuries_non_incapacitating    630078 non-null  object  
 3   crash_record_id                631405 non-null  object  
 4   injuries_incapacitating        630078 non-null  object  
 5   injuries_no_indication         630078 non-null  object  
 6   latitude                       629625 non-null  object  
 7   street_no                      631405 non-null  object  
 8   injuries_unknown               630078 non-null  object  
 9   device_condition               631405 non-null  object  
 10  crash_date                     631405 non-null  object  
 11  trafficway_type                631405 non-null  object  
 12  traffic_

In [16]:
#save this crashes_df with missing POINTs generated as geojson
crashes_df.to_file("data/crashes_cleaned.geojson", driver='GeoJSON')

/home/colin/anaconda3/envs/geospatial/lib/python3.8/site-packages/geopandas/io/file.py:234: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [17]:
crashes_df.set_crs(crs='EPSG:4326', inplace=True)

,geometry,injuries_fatal,injuries_non_incapacitating,crash_record_id,injuries_incapacitating,injuries_no_indication,latitude,street_no,injuries_unknown,device_condition,...,first_crash_type,injuries_reported_not_evident,most_severe_injury,prim_contributory_cause,sec_contributory_cause,street_direction,posted_speed_limit,injuries_total,lane_cnt,full_address
0,POINT (-87.71064 41.87352),None,None,0edc78f89df5b72ddaa1c1f567e229ad39c0e098953a9d...,None,None,41.873520168,3357,None,NO CONTROLS,...,PARKED MOTOR VEHICLE,None,None,FAILING TO REDUCE SPEED TO AVOID CRASH,NOT APPLICABLE,W,30,None,None,3357 W HARRISON ST CHICAGO IL
1,POINT (-87.67546 41.83184),0,0,49ff3f04d16f5a71e6d66436c94f6acfd939c20d6c3651...,0,2,41.831835811,3410,0,FUNCTIONING PROPERLY,...,REAR END,0,NO INDICATION OF INJURY,FOLLOWING TOO CLOSELY,NOT APPLICABLE,S,30,0,None,3410 S DAMEN AVE CHICAGO IL
2,POINT (-87.63002 41.83837),None,None,e580e89f187525bf685101a36fc64df499a72be926d5a9...,None,None,41.838371536,130,None,FUNCTIONING PROPERLY,...,ANGLE,None,None,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,W,35,None,None,130 W 31ST ST CHICAGO IL
3,POINT (-87.69494 41.86148),0,0,cdd7c5d90668e4d1bda12805ad19cec305667643a88806...,0,1,41.861481407,2747,0,NO CONTROLS,...,PARKED MOTOR VEHICLE,0,NO INDICATION OF INJURY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,30,0,None,2747 W OGDEN AVE CHICAGO IL
4,POINT (-87.64642 41.83796),0,0,e7c26ad2dd7f250b14acafe2d86265616f59f7c642a505...,0,2,41.837964154,3100,0,FUNCTIONING PROPERLY,...,TURNING,0,NO INDICATION OF INJURY,DISREGARDING TRAFFIC SIGNALS,DISREGARDING TRAFFIC SIGNALS,S,35,0,None,3100 S HALSTED ST CHICAGO IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639248,POINT (-87.72447 41.83589),0,0,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,0,2,41.835886103,3132,0,NO CONTROLS,...,SIDESWIPE SAME DIRECTION,0,NO INDICATION OF INJURY,IMPROPER OVERTAKING/PASSING,NOT APPLICABLE,S,30,0,None,3132 S PULASKI RD CHICAGO IL
639249,POINT (-87.70114 41.88402),0,0,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,0,2,41.884016475,199,0,FUNCTIONING PROPERLY,...,TURNING,0,NO INDICATION OF INJURY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,N,30,0,None,199 N SACRAMENTO BLVD CHICAGO IL
639250,POINT (-87.56195 41.76071),0,1,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,0,1,41.760710194,7400,0,NO CONTROLS,...,ANGLE,0,NONINCAPACITATING INJURY,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,S,30,1,None,7400 S EXCHANGE AVE CHICAGO IL
639251,POINT (-87.63876 41.88561),0,0,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,0,2,41.885609917,415,0,NO CONTROLS,...,PARKED MOTOR VEHICLE,0,NO INDICATION OF INJURY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,30,0,None,415 W LAKE ST CHICAGO IL


In [ ]:
print(crashes_df.crs)

In [ ]:
#cleaning lane count - jk, turns out when you call from the API, there's no 2 & 2.0, 4 & 4.0 problem
crashes_df['lane_cnt'].value_counts()

In [ ]:
#cleaning crash_date
test_time = crashes_df.loc[15,'crash_date']
type(test_time)
#it's a string so convert to datetime

In [ ]:
#try it on one single value
timestamp=pd.to_datetime(test_time, format = '%Y/%m/%d %H:%M:%S')

In [ ]:
#gives us a Timestamp
type(timestamp)
timestamp

In [ ]:
#can use Timestamp methods on it
#Timestamp is pandas equivalent of Python datetime 
#https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html
timestamp.month_name()

## making this a Timestamp screwed up my folium map. folium map wants a string
### having as Timestamp throws TypeError: Object of type Timestamp is not JSON serializable
#convert crash_date of whole df
#crashes_df['crash_date'] = pd.to_datetime(crashes_df['crash_date'], format = '%Y/%m/%d %H:%M:%S')

In [ ]:
#crash day of week, crash hour and crash month look fine as far as their range
crashes_df['crash_day_of_week'].value_counts()

In [ ]:
#injury columns to integers
#leave posted_speed_limit, lane_cnt as objects, they are more like categories than numerical measures
#leave crash_month, crash_day_of_week and crash_hour as objects for now, not sure what to make them

In [ ]:
crashes_df['injuries_fatal'] = pd.to_numeric(crashes_df['injuries_fatal'])
crashes_df['injuries_non_incapacitating'] = pd.to_numeric(crashes_df['injuries_non_incapacitating'])
crashes_df['injuries_no_indication'] = pd.to_numeric(crashes_df['injuries_no_indication'])
crashes_df['injuries_incapacitating'] = pd.to_numeric(crashes_df['injuries_incapacitating'])
crashes_df['injuries_unknown'] = pd.to_numeric(crashes_df['injuries_unknown'])
crashes_df['injuries_total'] = pd.to_numeric(crashes_df['injuries_total'])
crashes_df['injuries_reported_not_evident'] = pd.to_numeric(crashes_df['injuries_reported_not_evident'])

In [ ]:
crashes_df.dtypes

## get pedestrians from the people API, clean

In [ ]:
endpoint_people = 'https://data.cityofchicago.org/resource/u6pd-qa9d.json?person_type=PEDESTRIAN&$limit=16000'
res = requests.get(endpoint_people)
res = res.json()
peds_df = pd.DataFrame(res)

In [ ]:
peds_df.info()

In [ ]:
#remove peds who were impaired by drugs or alcohol or both
#DON'T filter in ['NORMAL','UNKNOWN','REMOVED BY EMS','OTHER','EMOTIONAL'] b/c you lose 1000 NaNs 
#instead, filter OUT with impaired_list and ~

impaired_list = ['IMPAIRED - ALCOHOL',
'HAD BEEN DRINKING',
'IMPAIRED - DRUGS',
'IMPAIRED - ALCOHOL AND DRUGS',
'FATIGUED/ASLEEP',
'ILLNESS/FAINTED',
'MEDICATED']
peds_df = peds_df.loc[~peds_df['physical_condition'].isin(impaired_list)]

## generate 4 dfs for peds by injury classification

In [ ]:
peds_fatal_df = peds_df.loc[peds_df['injury_classification'] == 'FATAL']
peds_fatal_df.shape

In [ ]:
peds_incapacitating_df = peds_df.loc[peds_df['injury_classification'] == 'INCAPACITATING INJURY']
peds_incapacitating_df.shape

In [ ]:
peds_nonincapacitating_df = peds_df.loc[peds_df['injury_classification'] == 'NONINCAPACITATING INJURY']
peds_nonincapacitating_df.shape

In [ ]:
peds_no_signif_injury_df = peds_df.loc[peds_df['injury_classification'].isin(['REPORTED, NOT EVIDENT','NO INDICATION OF INJURY'])]
peds_no_signif_injury_df.shape

## extract crash_record_ids from peds_dfs and filter crashes_df to create 4 dfs for crashes by injury classification
### didn't do nonincapacitating or no signif injury yet
### there could actually be duplicate geometries here b/c 2 peds in people would have same crash_record_id if there were 2 peds in crash

In [ ]:
crash_id_peds_fatal = peds_fatal_df['crash_record_id'].tolist()
crashes_ped_fatal_df = crashes_df.loc[crashes_df['crash_record_id'].isin(crash_id_peds_fatal)].reset_index(drop=True)
crashes_ped_fatal_df.info()

In [ ]:
crashes_ped_fatal_df.plot();

In [ ]:
crash_id_peds_incapacitating = peds_incapacitating_df['crash_record_id'].tolist()
crashes_ped_incapacitating_df = crashes_df.loc[crashes_df['crash_record_id'].isin(crash_id_peds_incapacitating)].reset_index(drop=True)
crashes_ped_incapacitating_df.info()

## map things to do: clean up crash_date string
## merge columns like pedpedal_location and pedpedal_action from people to be able to display on map popup

In [ ]:
area_center = [41.881288,-87.686729]
ped_fatal_cluster_map = folium.Map(location =  area_center, zoom_start = 11)
marker_cluster = MarkerCluster().add_to(ped_fatal_cluster_map)
#folium.GeoJson(crashes_fatal_df).add_to(fatal_cluster_map)

for row_index, row_values in crashes_ped_fatal_df.iterrows():
    loc = (row_values['geometry'].y, row_values['geometry'].x)
    pop = row_values['trafficway_type'], row_values['traffic_control_device'],row_values['crash_date']
    icon=folium.Icon(color="blue",icon='user', prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon)
    
    marker.add_to(marker_cluster)


ped_fatal_cluster_map

In [ ]:
area_center = [41.881288,-87.686729]
ped_incapacitating_cluster_map = folium.Map(location =  area_center, zoom_start = 11)
marker_cluster = MarkerCluster().add_to(ped_incapacitating_cluster_map)
#folium.GeoJson(crashes_fatal_df).add_to(fatal_cluster_map)

for row_index, row_values in crashes_ped_incapacitating_df.iterrows():
    loc = (row_values['geometry'].y, row_values['geometry'].x)
    pop = row_values['trafficway_type'], row_values['traffic_control_device'],row_values['crash_date']
    icon=folium.Icon(color="blue",icon='user', prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon)
    
    marker.add_to(marker_cluster)


ped_incapacitating_cluster_map

#split POINT geometries into separate lat and long columns
ped_crashes_df['lon'] = ped_crashes_df.geometry.apply(lambda p: p.x)
ped_crashes_df['lat'] = ped_crashes_df.geometry.apply(lambda p: p.y)

## get cyclists from the people API, clean

In [ ]:
endpoint_people = 'https://data.cityofchicago.org/resource/u6pd-qa9d.json?person_type=BICYCLE&$limit=16000'
res = requests.get(endpoint_people)
res = res.json()
cyclists_df = pd.DataFrame(res)

In [ ]:
cyclists_df.info()

In [ ]:
#remove cyclists who were impaired by drugs or alcohol or both
physical_condition_list = ['NORMAL','UNKNOWN','REMOVED BY EMS','OTHER','EMOTIONAL']
cyclists_df = cyclists_df.loc[cyclists_df['physical_condition'].isin(physical_condition_list)]

In [ ]:
cyclists_df['injury_classification'].value_counts()

## generate 4 dfs for cyclists by injury classification

In [ ]:
cyclists_fatal_df = cyclists_df.loc[cyclists_df['injury_classification'] == 'FATAL']
cyclists_fatal_df.shape

In [ ]:
cyclists_incapacitating_df = cyclists_df.loc[cyclists_df['injury_classification'] == 'INCAPACITATING INJURY']
cyclists_incapacitating_df.shape

In [ ]:
cyclists_nonincapacitating_df = cyclists_df.loc[cyclists_df['injury_classification'] == 'NONINCAPACITATING INJURY']
cyclists_nonincapacitating_df.shape

In [ ]:
cyclists_no_signif_injury_df = cyclists_df.loc[cyclists_df['injury_classification'].isin(['REPORTED, NOT EVIDENT','NO INDICATION OF INJURY'])]
cyclists_no_signif_injury_df.shape

## extract crash_record_ids from cyclists_dfs and filter crashes_df to create 4 dfs for crashes by injury classification

In [ ]:
crash_id_cyclists_fatal = cyclists_fatal_df['crash_record_id'].tolist()
crashes_cyclist_fatal_df = crashes_df.loc[crashes_df['crash_record_id'].isin(crash_id_cyclists_fatal)].reset_index(drop=True)
crashes_cyclist_fatal_df.info()

In [ ]:
#why does this go down from 854 to 834?
crash_id_cyclists_incapacitating = cyclists_incapacitating_df['crash_record_id'].tolist()
crashes_cyclist_incapacitating_df = crashes_df.loc[crashes_df['crash_record_id'].isin(crash_id_cyclists_incapacitating)].reset_index(drop=True)
crashes_cyclist_incapacitating_df.info()

In [ ]:
area_center = [41.881288,-87.686729]
cyclist_fatal_cluster_map = folium.Map(location =  area_center, zoom_start = 11)
marker_cluster = MarkerCluster().add_to(cyclist_fatal_cluster_map)
#folium.GeoJson(crashes_fatal_df).add_to(fatal_cluster_map)

for row_index, row_values in crashes_cyclist_fatal_df.iterrows():
    loc = (row_values['geometry'].y, row_values['geometry'].x)
    pop = row_values['trafficway_type'], row_values['traffic_control_device'],row_values['crash_date']
    icon=folium.Icon(color="blue",icon='user', prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon)
    
    marker.add_to(marker_cluster)


cyclist_fatal_cluster_map